In [2]:

import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
import datetime
'''
REDIS_HOST = 'redis-13648.c267.us-east-1-4.ec2.cloud.redislabs.com'
REDIS_PORT = 13648
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'HqhFx8hRnbywxQQHc7XqFi7n814qF3Sj'
'''
REDIS_HOST = 'redis-11633.c293.eu-central-1-1.ec2.cloud.redislabs.com' 
REDIS_PORT = 11633
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'ohqqd7LasHtDIbzLOt00SHV393FUhATt'
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
global bds
bds = []
# endpoint /status
class Status(object):
    exposed = True
    def GET(self, *path, **query):
        response_dict = {
            'status': 'online'
        }
        response = json.dumps(response_dict)

        return response

##device list
##endpoints 1

# CherryPy endpoint for retrieving device list

class DeviceList(object):
    exposed = True

    def GET(self, *path, **query):
        blt = query.get('blt', None)
        plugged = query.get('plugged', None)

        # Validate query parameters
        if blt is not None:
            try:
                blt = int(blt)
                if not 0 <= blt <= 100:
                    raise ValueError
            except ValueError:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid battery threshold value')

        if plugged is not None:
            try:
                plugged = int(plugged)
                if plugged not in [0, 1]:
                    raise ValueError
            except ValueError:
                raise cherrypy.HTTPError(400, 'Bad Request: invalid plugged value')

        # Retrieve and filter devices from Redis
        device_keys = redis_client.keys("*:power")
        print(device_keys)  # Use the identified keys
        mac_addresses = []

        for key in device_keys:
            decoded_key = key.decode()
            mac_address = decoded_key.split(':')[0]
            try:
                # Retrieve TimeSeries data for 'power' and 'battery' keys
                device_data_power = redis_client.ts().range(f'{mac_address}:power', "-", "+")
                device_data_battery = redis_client.ts().range(f'{mac_address}:battery', "-", "+")
                
            except redis.exceptions.ResponseError:
                # Handle non-JSON keys differently (if any)
                # For example, you might use redis_client.get(...) instead
                continue

            # Extract battery levels and power plugged information from TimeSeries data
            battery_levels = [entry[1] for entry in device_data_battery]
            power_plugged = [entry[1] for entry in device_data_power]
            if ((blt is None or max(battery_levels) <= blt) and
                    (plugged is None or  plugged in power_plugged)):
                mac_addresses.append(mac_address)
        mac_addresses =list(set(mac_addresses))
        response = {'mac_addresses': mac_addresses}
        print(response)  # Print the response for verification
        return json.dumps(response)

##devicebatterystatus
##endpoints 2
##endoints 3
class DeviceBatteryStatus(object):
    exposed = True

    def GET(self, *path, **query):
        # Extracting variables from the path
        if len(path) < 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]

        # Extracting variables from the query parameters
        start_date = query.get('start_date', None)
        end_date = query.get('end_date', None)

        # Validate the MAC address and date parameters
        if not mac_address:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')

        try:
            if start_date:
                start_date = datetime.datetime.fromisoformat(start_date)
            if end_date:
                end_date = datetime.datetime.fromisoformat(end_date)
                if start_date and start_date >= end_date:
                    raise ValueError
        except ValueError:
            raise cherrypy.HTTPError(400, 'Bad Request: invalid date format or range')

        # Retrieve data from Redis for the specified keys
        device_data_power = redis_client.ts().range(f'{mac_address}:power', "-", "+")
        device_data_battery = redis_client.ts().range(f'{mac_address}:battery', "-", "+")

        timestamps = []
        battery_levels = []
        power_plugged = []

        for (timestamp_battery, battery_level), (timestamp_power, power_status) in zip(device_data_battery, device_data_power):
              timestamp_seconds = timestamp_battery / 1000
              date_obj = datetime.datetime.fromtimestamp(timestamp_seconds)

              if start_date <= date_obj <= end_date:
                   timestamps.append(timestamp_battery)
                   battery_levels.append(battery_level)
                   power_plugged.append(power_status)
        print(battery_levels)
        # Format the response
        response = {
            'mac_address': mac_address,
            'timestamps': timestamps,
            'battery_levels': battery_levels,
            'power_plugged': power_plugged
        }

        return json.dumps(response)
    def DELETE(self, *path, **query):
        # Extracting variables from the path
        if len(path) < 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        print(mac_address)

        if not mac_address:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')

        # Define the key pattern for the device's timeseries data
        device_key = f'{mac_address}:battery'

        # Check if the device exists in Redis
        exists = redis_client.exists(device_key)
        if not exists:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')

        # Delete the timeseries data associated with the MAC address
        redis_client.delete(device_key)

        # Returning a success response
        return json.dumps({'status': 'success', 'message': 'Timeseries data deleted'})

if __name__ == '__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
    
    # Mount the new endpoints
    cherrypy.tree.mount(Status(), '/status', conf)
    cherrypy.tree.mount(DeviceList(), '/devices', conf)  # Updated endpoint path
    cherrypy.tree.mount(DeviceBatteryStatus(), '/device', conf)
 

    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[01/Feb/2024:14:55:05] ENGINE Bus STARTING
[01/Feb/2024:14:55:05] ENGINE Started monitor thread 'Autoreloader'.
[01/Feb/2024:14:55:05] ENGINE Serving on http://0.0.0.0:8080
[01/Feb/2024:14:55:05] ENGINE Bus STARTED
[b'0xf14d2b988308:power', b'0xc22e0e2f1be3:power', b'0x8c8590ce32bc:power', b'0xb4b686bb4c52:power']
{'mac_addresses': ['0xf14d2b988308', '0xb4b686bb4c52', '0x8c8590ce32bc']}
127.0.0.1 - - [01/Feb/2024:14:55:15] "GET /devices HTTP/1.1" 200 73 "" "python-requests/2.28.1"
[50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0, 75.0]
127.0.0.1 - - [01/Feb/2024:14:55:15] "GET /device/0xf14d2b988308?start_date=2024-01-10+00%3A00%3A00&end_date=2024-02-03+00%3A00%3A00 HTTP/1.1" 200 1128 "" "python-requests/2.28.1"
[42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42.0, 42

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3de268ea-71d7-469a-8d76-c287e5ad6570' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>